# Basic retreival

planning

1. pdf loader
2. vector store (FAISS)
3. generate answer with sources


Further:
- inline citation
- conversational


In [16]:
import pprint

In [6]:
# load credentials
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [9]:
# load pdf and chunk it
import os
from langchain_community.document_loaders import PyPDFLoader

documents = []
pdf_dir = os.path.join(os.getcwd(),"data")
for filename in os.listdir(pdf_dir):
    if filename.endswith('.pdf'):
      loader = PyPDFLoader(os.path.join(pdf_dir, filename))
      pages = loader.load_and_split()
      documents.extend(pages)
print("Number of document chunks:", documents)


Number of document chunks: [Document(page_content='Enabling Large Language Models to Generate Text with Citations\nTianyu Gao Howard Yen Jiatong Yu Danqi Chen\nDepartment of Computer Science & Princeton Language and Intelligence\nPrinceton University\n{tianyug,hyen,jiatongy,danqic}@cs.princeton.edu\nAbstract\nLarge language models (LLMs) have emerged\nas a widely-used tool for information seeking,\nbut their generated outputs are prone to halluci-\nnation. In this work, our aim is to allow LLMs\nto generate text with citations , improving their\nfactual correctness and verifiability. Existing\nwork mainly relies on commercial search en-\ngines and human evaluation, making it chal-\nlenging to reproduce and compare different\nmodeling approaches. We propose ALCE , the\nfirst benchmark for Automatic LLMs’ Citation\nEvaluation. ALCE collects a diverse set of\nquestions and retrieval corpora and requires\nbuilding end-to-end systems to retrieve sup-\nporting evidence and generate answers w

In [10]:
# create vector index

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vector = FAISS.from_documents(documents, OpenAIEmbeddings())

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(ChatOpenAI(), prompt)

In [13]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [19]:
question = "what is Automatic LLMs’ Citation Evaluation?"
response = retrieval_chain.invoke({"input": question})
pprint.pp(response["answer"])

('Automatic LLMs’ Citation Evaluation (ALCE) is a benchmark proposed in the '
 'provided context that collects a diverse set of questions and retrieval '
 'corpora and requires building end-to-end systems to retrieve supporting '
 'evidence and generate answers with citations. ALCE aims to evaluate the '
 'fluency, correctness, and citation quality of text generated by Large '
 'Language Models (LLMs) automatically, using metrics that have been '
 'demonstrated to have strong correlation with human judgments.')


In [15]:
import re
count = 1
for doc in response["context"]:
  path = f'{doc.metadata["source"]}'
  # Regex pattern
  pattern = r'[^/]+$'

  # Extract the last part after the last slash
  filename = re.search(pattern, path)

  if filename:
    print(f'[{count}]: {filename.group()}, pg {doc.metadata["page"]}')
  count+=1


[1]: alce.pdf, pg 0
[2]: attribute firtst then generate.pdf, pg 4
[3]: alce.pdf, pg 8
[4]: training language models to generate text with citations via fine grained rewards.pdf, pg 0
